# Titanic Survival Prediction - Model Building

In this notebook, we focus on building a predictive model for the Titanic dataset.  
Our goal is to create a machine learning pipeline that handles data preprocessing (imputation, encoding, scaling) and trains a model to predict passenger survival.

For now, we will directly work with the raw training data.  
A detailed Exploratory Data Analysis (EDA) will be performed separately in a different file.

## This notebook covers:

- Initial basic cleaning for model needs
- Train-validation split
- Building preprocessing steps (inside a pipeline)
- Training different models
- Evaluating performance

Once the best model is finalized, it will be used to make predictions on the unseen test data.

## 1. Importing Required Libraries

In [132]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn import set_config

## 2. Loading Dataset

In [133]:
df = pd.read_csv("data/train.csv")

In [134]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [136]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [137]:
df.duplicated().sum()

np.int64(0)

## 3. Basic Data Cleaning

In [138]:
# Removing unnecessary columns
df.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1, inplace=True)

In [139]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


## 4. Train-Test Split

In [140]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived'])
                                                    , df['Survived'], 
                                                    test_size=0.2, 
                                                    random_state=42)

In [141]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


## 5. Building Pipeline

In [142]:
# Imputing missing values
num_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer()), 
    #("scaler", StandardScaler())
]) 

# Imputing missing values and Encoding Categorical data
categ_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy='most_frequent')), 
    ("ohe", OneHotEncoder(sparse_output=False, handle_unknown="ignore"))
])


# Preprocessing using column transformer
preprocessor = ColumnTransformer([
    ("num", num_pipeline, [2,5]), 
    ("categ", categ_pipeline, [1, 6]) 
],remainder='passthrough')


#Final pipeline
pipe = Pipeline(steps=[
    ("preprocessing", preprocessor), 
    ("classifier",RandomForestClassifier(n_estimators=100, random_state=42))
])

## 6. Training Model

In [143]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  [2, 5]),
                                                 ('categ',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  [1, 6])])),
                ('classifier', RandomForestClassifier(random_state=42))])

## 7. Model Evaluation

In [144]:
# Predict
y_pred = pipe.predict(X_test)

In [145]:
# Prediction accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred) #0.8268156424581006

0.8212290502793296

In [146]:
# Checking which features contribute the most to the prediction

model = pipe.named_steps['classifier']
importances = model.feature_importances_
print(importances)

[0.24892882 0.25727197 0.16348797 0.12184834 0.01185911 0.00943428
 0.01567752 0.08707972 0.04888006 0.0355322 ]


## 8. Cross Validation

In [154]:
from sklearn.model_selection import cross_val_score

X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')

print("Mean accuracy:", np.mean(scores))

Mean accuracy: 0.8182160567447117
